<a href="https://colab.research.google.com/github/jessecanada/MAPS/blob/master/3.%20Azure_phenotype_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up Azure environtment

In [1]:
!pip -q install azure-cognitiveservices-vision-customvision
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

     |████████████████████████████████| 112kB 9.3MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 


In [0]:
from google.colab import files
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [0]:
ENDPOINT = "https://westus2.api.cognitive.microsoft.com/"

# PTEN_classification_RBG
training_key = "65c576c79b7d4365981393e84e79350e"
prediction_key = "bae40906a41d43d2836d6eedc23006ec"
prediction_resource_id = "/subscriptions/6c830677-7f0f-4065-9293-38e0e58684f6/resourceGroups/PTEN/providers/Microsoft.CognitiveServices/accounts/jesse_chao"
trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

publish_iteration_name = "Iteration2"

project = trainer.get_project(project_id='1eae5342-91d5-4f2c-9848-9652c1e13b36')

In [4]:
project.id # double check project ID to make sure it's correct

'1eae5342-91d5-4f2c-9848-9652c1e13b36'

In [0]:
trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)

## Get the files

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!mkdir C124R_singles_filtered

In [0]:
#!rmdir --ignore-fail-on-non-empty WT_filtered/

In [0]:
#!cp -r /content/drive/My\ Drive/PTEN_images/M134I_filtered /content/
#cp -r /content/drive/My\ Drive/PTEN_images/C124R_cluster3/*.jpg /content/C124R_singles_filtered/


In [0]:
#!mv /content/WT_singles_filtered/WT_all_cluster0/*.jpg WT_singles_filtered/
#!mv /content/WT_singles_filtered/WT_all_cluster1/*.jpg WT_singles_filtered/

In [40]:
!unzip /content/drive/My\ Drive/PTEN_images/Y180H_singles_RGB_manually_filtered.zip -d /content/

Archive:  /content/drive/My Drive/PTEN_images/Y180H_singles_RGB_manually_filtered.zip
   creating: /content/Y180H_singles_RGB_manually_filtered/
  inflating: /content/Y180H_singles_RGB_manually_filtered/merged_191024160002_A02f250_1.jpg  
   creating: /content/__MACOSX/Y180H_singles_RGB_manually_filtered/
  inflating: /content/__MACOSX/Y180H_singles_RGB_manually_filtered/._merged_191024160002_A02f250_1.jpg  
  inflating: /content/Y180H_singles_RGB_manually_filtered/merged_191024160001_A02f116_6.jpg  
  inflating: /content/__MACOSX/Y180H_singles_RGB_manually_filtered/._merged_191024160001_A02f116_6.jpg  
  inflating: /content/Y180H_singles_RGB_manually_filtered/merged_191024160002_A02f294_5.jpg  
  inflating: /content/__MACOSX/Y180H_singles_RGB_manually_filtered/._merged_191024160002_A02f294_5.jpg  
  inflating: /content/Y180H_singles_RGB_manually_filtered/merged_191024160002_A02f289_0.jpg  
  inflating: /content/__MACOSX/Y180H_singles_RGB_manually_filtered/._merged_191024160002_A02f289

In [41]:
# confirm how many cells are to be analyzed
!ls /content/Y180H_singles_RGB_manually_filtered/ | wc -l

156


## Azure classification predictions

### Testing codes. Make a df from 2 test predictions.

In [19]:
# first image and first set of predictions

with open('/content/WT_singles_filtered/10_03_19_PTEN_IF_WT_4_11.jpg', "rb") as image: # r = read, b = binary (eg. images)
    results = predictor.classify_image(project.id, publish_iteration_name, image)

    # get the prediction results
    for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
    
    print()
    
    probabilities = [prediction.probability*100 for prediction in results.predictions]
    col_names = [prediction.tag_name for prediction in results.predictions]

    df_test1 = pd.DataFrame([probabilities], columns = col_names)
    df_test1.insert(0, 'image_ID', 'test_img_1')

    print(df_test1)

	non_nuclear: 99.95%
	diffused: 0.05%
	nuclear: 0.00%

     image_ID  non_nuclear  diffused   nuclear
0  test_img_1     99.95431  0.045688  0.000012


In [20]:
# 2nd image and 2nd set of predictions

with open('/content/WT_singles_filtered/o-1bd92f1fdd1e4ba0817a8b9202dd39b9_41.jpg', "rb") as image: # r = read, b = binary (eg. images)
    results = predictor.classify_image(project.id, publish_iteration_name, image)

    # get the prediction results
    for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
    
    print()
    
    probabilities = [prediction.probability*100 for prediction in results.predictions]
    col_names = [prediction.tag_name for prediction in results.predictions]

    df_test2 = pd.DataFrame([probabilities], columns = col_names)
    df_test2.insert(0, 'image_ID', 'test_img_2')

    print(df_test2)

	diffused: 100.00%
	nuclear: 0.00%
	non_nuclear: 0.00%

     image_ID  diffused       nuclear   non_nuclear
0  test_img_2     100.0  4.037947e-10  2.850407e-14


In [21]:
# testing 1st method of combining df. 
# NOTE: this doesn't work when concatenating to an empty df,
# which is necessary when running it in a loop

print(pd.concat([df_test1, df_test2], sort=False))

     image_ID   non_nuclear    diffused       nuclear
0  test_img_1  9.995431e+01    0.045688  1.190392e-05
0  test_img_2  2.850407e-14  100.000000  4.037947e-10


In [38]:
test1_tags = df_test1.columns[1:]
test1_probs = df_test1.iloc[0, 1:].to_list()
test1_dict = dict(zip(test1_tags, test1_probs))

print(f'test1_dict = {test1_dict}', '\n')

temp_list = []
test1_list = []
for i in sorted(test1_dict):
  test1_list.append(test1_dict[i])
test1_list.insert(0, df_test1.iloc[0,0])

temp_list.append(test1_list)
print(f'temp_list = {temp_list}')

test1_dict = {'non_nuclear': 99.95431, 'diffused': 0.0456881244, 'nuclear': 1.19039242e-05} 

temp_list = [['test_img_1', 0.0456881244, 99.95431, 1.19039242e-05]]


In [47]:
# make a loop
temp_list = []

for i, df in enumerate([df_test1, df_test2]):
  tags = df.columns[1:]
  probs = df.iloc[0, 1:].to_list()
  
  test_dict = dict(zip(tags, probs))
  test_list = []
  for i in sorted(test_dict):
    test_list.append(test_dict[i])
  test_list.insert(0, df.iloc[0,0])

  temp_list.append(test_list)

print(temp_list)

[['test_img_1', 0.0456881244, 99.95431, 1.19039242e-05], ['test_img_2', 100.0, 2.8504074800000003e-14, 4.03794663e-10]]


In [53]:
list_of_tests = ['/content/WT_singles_filtered/10_03_19_PTEN_IF_WT_4_11.jpg',\
                 '/content/WT_singles_filtered/o-1bd92f1fdd1e4ba0817a8b9202dd39b9_41.jpg',\
                 '/content/WT_singles_filtered/10_28_19_PTEN_IF_WT_9_3.jpg']

temp_list = []
for img_path in list_of_tests:
  with open(img_path, "rb") as image: # r = read, b = binary (eg. images)
    image_ID = img_path[-10:-4]
    results = predictor.classify_image(project.id, publish_iteration_name, image)
    
    for prediction in results.predictions:
      print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
      
    print()

    tags = [prediction.tag_name for prediction in results.predictions]
    probabilities = [prediction.probability*100 for prediction in results.predictions]
    predictions_dict = dict(zip(tags, probabilities))
    predictions_list = [predictions_dict[i] for i in sorted(predictions_dict)]
    predictions_list.insert(0, image_ID)
    temp_list.append(predictions_list)

print(temp_list)

	non_nuclear: 99.95%
	diffused: 0.05%
	nuclear: 0.00%

	diffused: 100.00%
	nuclear: 0.00%
	non_nuclear: 0.00%

	non_nuclear: 100.00%
	diffused: 0.00%
	nuclear: 0.00%

[['T_4_11', 0.0456881244, 99.95431, 1.19039242e-05], ['9b9_41', 100.0, 2.8504074800000003e-14, 4.03794663e-10], ['WT_9_3', 1.3272584400000001e-09, 100.0, 5.21619367e-20]]


In [59]:
col_names = [i for i in sorted(predictions_dict)]
col_names.insert(0, 'image_ID')
df_temp = pd.DataFrame(temp_list, columns = col_names)
print(df_temp)

  image_ID      diffused   non_nuclear       nuclear
0   T_4_11  4.568812e-02  9.995431e+01  1.190392e-05
1   9b9_41  1.000000e+02  2.850407e-14  4.037947e-10
2   WT_9_3  1.327258e-09  1.000000e+02  5.216194e-20


### make predictions

In [42]:
wrk_dir = '/content/Y180H_singles_RGB_manually_filtered/'
temp_list = []

for entry in os.scandir(wrk_dir):
  if entry.name.endswith('.jpg'):
    image_ID = entry.name[:-4]
    print(f'image_ID: {image_ID}')

    # open an image and get back the prediction results
    with open(wrk_dir+entry.name, mode="rb") as image: #'r'-read, 'b'-binary (for images)
      results = predictor.classify_image(project.id, publish_iteration_name, image)
    
      # get the results
      tags = [prediction.tag_name for prediction in results.predictions]
      probabilities = [prediction.probability*100 for prediction in results.predictions]
      # make a dictionary of tag:prob pairs
      predictions_dict = dict(zip(tags, probabilities))
      # sort the tags in alphabetical order, append the corresponding prob of the sorted tags
      predictions_list = [predictions_dict[i] for i in sorted(predictions_dict)]
      # add image_ID to the beginning of the list
      predictions_list.insert(0, image_ID)
      # append the sorted list to a list as a compound list
      temp_list.append(predictions_list)
    
      for i in sorted(predictions_dict) : 
        print(f'{i}: {predictions_dict[i]}:.2f') 
      print()

image_ID: merged_191024160001_A02f100_2
diffused: 1.06238875e-06:.2f
non_nuclear: 100.0:.2f
nuclear: 1.76529166e-31:.2f

image_ID: merged_191024160002_A02f250_4
diffused: 0.38223352699999996:.2f
non_nuclear: 99.61777000000001:.2f
nuclear: 1.41995936e-13:.2f

image_ID: merged_191024160002_A02f249_5
diffused: 99.9204338:.2f
non_nuclear: 0.0119858538:.2f
nuclear: 0.06758246450000001:.2f

image_ID: merged_191024160002_A02f262_7
diffused: 99.99997619999999:.2f
non_nuclear: 2.20268831e-05:.2f
nuclear: 1.00542645e-15:.2f

image_ID: merged_191024160002_A02f294_12
diffused: 0.0019529460000000001:.2f
non_nuclear: 99.998045:.2f
nuclear: 2.8986516299999996e-15:.2f

image_ID: merged_191024160002_A02f275_2
diffused: 100.0:.2f
non_nuclear: 4.500061e-06:.2f
nuclear: 2.62980924e-13:.2f

image_ID: merged_191024160002_A02f289_0
diffused: 0.00246070158:.2f
non_nuclear: 99.9975443:.2f
nuclear: 3.6422344100000003e-20:.2f

image_ID: merged_191024160002_A02f262_0
diffused: 1.77318616e-33:.2f
non_nuclear: 100.

In [35]:
print(temp_list[:20])

[['merged_191120110001_C02f12_1', 28.862401799999997, 2.97397e-10, 71.1375952], ['merged_191120110001_C02f237_4', 99.9309063, 0.0690494431, 3.662342e-05], ['merged_191120110001_C02f237_2', 2.45327869e-08, 1.0833777200000001e-32, 100.0], ['merged_191120110001_C02f72_4', 3.10186834e-05, 98.02455, 1.97542589], ['merged_191120110001_C02f107_5', 2.3687923499999998e-12, 0.0069846579999999995, 99.9930143], ['merged_191120110001_C02f94_2', 3.16279353e-19, 0.0, 100.0], ['merged_191120110001_C02f98_1', 3.8622509999999997, 96.13775, 1.55244741e-14], ['merged_191120110001_C02f236_1', 8.67026448e-12, 100.0, 2.9675736400000004e-15], ['merged_191120110001_C02f34_1', 0.01065261, 0.06286259999999999, 99.92649], ['merged_191120110001_C02f213_1', 2.5950720000000004e-11, 100.0, 1.81696576e-28], ['merged_191120110001_C02f134_3', 2.635391e-07, 100.0, 1.5214220000000001e-09], ['merged_191120110001_C02f157_0', 0.12240227099999999, 80.5704236, 19.3071783], ['merged_191120110001_C02f117_2', 5.4394051799999996e-

In [43]:
col_names = [i for i in sorted(predictions_dict)]
col_names.insert(0, 'image_ID')
df_predict = pd.DataFrame(temp_list, columns = col_names)
df_predict.head(10)

,image_ID,diffused,non_nuclear,nuclear
0,merged_191024160001_A02f100_2,1.062389e-06,100.000000,1.765292e-31
1,merged_191024160002_A02f250_4,3.822335e-01,99.617770,1.419959e-13
2,merged_191024160002_A02f249_5,9.992043e+01,0.011986,6.758246e-02
3,merged_191024160002_A02f262_7,9.999998e+01,0.000022,1.005426e-15
4,merged_191024160002_A02f294_12,1.952946e-03,99.998045,2.898652e-15
5,merged_191024160002_A02f275_2,1.000000e+02,0.000005,2.629809e-13
6,merged_191024160002_A02f289_0,2.460702e-03,99.997544,3.642234e-20
7,merged_191024160002_A02f262_0,1.773186e-33,100.000000,2.421084e-30
8,merged_191024160002_A02f298_6,7.095198e-05,7.213560,9.278637e+01
9,merged_191024160002_A02f310_6,3.816229e+00,96.183770,2.202939e-11


In [44]:
len(df_predict) # does it match the number of images in the folder?

156

In [0]:
#type(df_predict.iloc[0, 1])

numpy.float64

In [0]:
#df_predict = df_predict.apply(pd.to_numeric, errors='ignore') # convert whole df from str ot int, ignore errors

In [45]:
df_predict['nuclear'][df_predict['nuclear'] > 50].count()

24

In [0]:
df_predict.to_csv('Y180H_classification_RGB_iter2.csv', index=False)